In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from matplotlib import pyplot as plt
import scvelo as scv
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import scipy
from scipy import io
from matplotlib import rcParams

In [ ]:
sc.settings.verbosity=3
sc.logging.print_header()
sc.set_figure_params(dpi=300, fontsize=10)
FIGSIZE = (4, 3)
rcParams["figure.figsize"] = FIGSIZE

In [ ]:
oviductepithelial = sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/oviductepitheliumforconcat.h5ad')
oviductepithelial

In [ ]:
menstrualtissuedict = {
 'Secretory_FT': ['Secretory'],
'Proliferative_FT':['Proliferative'],
 'Post-menopausal_FT': ['Post-menopausal'], 
 'Peri-menopausal_FT': ['Peri-menopausal']}

oviductepithelial.obs['menstrual_status_tissue'] = np.nan

for i in menstrualtissuedict.keys():
    ind = pd.Series(oviductepithelial.obs.menstrual_status).isin(menstrualtissuedict[i])
    oviductepithelial.obs.loc[ind,'menstrual_status_tissue'] = i

oviductepithelial.obs['menstrual_status_tissue']

In [ ]:
sc.pl.umap(oviductepithelial, color=['epithelial_celltypes'])

In [ ]:
menstrual=oviductepithelial[oviductepithelial.obs['menstrual_status'].isin(['Proliferative','Secretory']),:].copy()
grouped = menstrual.obs.groupby('menstrual_status')
counts = grouped['epithelial_celltypes'].value_counts().unstack().fillna(0)
proportions = counts.divide(counts.sum(axis=1), axis=0)

counts

In [ ]:
grouped = menstrual.obs.groupby('menstrual_status')
counts = grouped['epithelial_celltypes'].value_counts().unstack().fillna(0)
proportions = counts.divide(counts.sum(axis=1), axis=0)


custom_palette = sns.color_palette('Set3',12)
sns.set_palette('tab10',12)

plt.figure(figsize=(6,6))
proportions.plot(kind='barh', stacked=True, )
plt.xlabel('Proportion')
plt.ylabel('Menstrual Status')
plt.title('Proportion of Cell Types Across Menstrual Status')
plt.legend(loc='center left',bbox_to_anchor=(1.0,0.5))
plt.tight_layout()
sns.reset_defaults()
plt.show()

In [ ]:
secretory=oviductepithelial[oviductepithelial.obs['menstrual_status'].isin(['Secretory']),:].copy()
proliferative=oviductepithelial[oviductepithelial.obs['menstrual_status'].isin(['Proliferative']),:].copy()

In [ ]:
endo= sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/endometrialepithelialclusteringfinalforconcat.h5ad')
endo.obs

In [ ]:
menstrualtissuedictendo = {
 'Secretory_endo': ['Secretory'],
'Proliferative_endo':['Proliferative']}

endo.obs['menstrual_status_tissue'] = np.nan

for i in menstrualtissuedictendo.keys():
    ind = pd.Series(endo.obs.menstrual_status).isin(menstrualtissuedictendo[i])
    endo.obs.loc[ind,'menstrual_status_tissue'] = i

endo.obs['menstrual_status_tissue']

In [ ]:
grouped = endo.obs.groupby('menstrual_status')
counts = grouped['epithelial_celltypes'].value_counts().unstack().fillna(0)
proportions = counts.divide(counts.sum(axis=1), axis=0)

counts

In [ ]:
grouped = endo.obs.groupby('menstrual_status')
counts = grouped['epithelial_celltypes'].value_counts().unstack().fillna(0)
proportions = counts.divide(counts.sum(axis=1), axis=0)

custom_palette = sns.color_palette('Set3',12)
sns.set_palette('tab10',12)

plt.figure(figsize=(6,10))
proportions.plot(kind='barh', stacked=True)
plt.xlabel('Proportion')
plt.ylabel('Menstrual Status')
plt.title('Proportion of Cell Types Across Menstrual Status')
plt.legend(loc='center left',bbox_to_anchor=(1.0,0.5))
plt.tight_layout()
sns.reset_defaults()
plt.show()

In [ ]:
sc.pl.umap(endo, color='menstrual_status')

In [ ]:
endosecretory=endo[endo.obs['menstrual_status'].isin(['Secretory']),:].copy()

In [ ]:
oviendo = sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/oviendoumapallphase.h5ad')

In [ ]:
oviendo.obs['menstrual_status_tissue']

In [ ]:
secretoryoviendo= oviendo[oviendo.obs['menstrual_status_tissue'].isin(['Secretory_FT','Secretory_endo']),:].copy()
results_file = '/home/j87832lw/mounting/oviduct/data/secretoryendoonlyoviendoumap.h5ad'
secretoryoviendo.write(results_file)

In [ ]:
secretoryoviendo = sc.read_h5ad('/home/j87832lw/mounting/oviduct/data/secretoryendoonlyoviendoumap.h5ad')
sc.pl.umap(secretoryoviendo, color=['broad_celltypes'] )

In [ ]:
sc.pp.log1p(secretoryoviendo)
sc.tl.rank_genes_groups(secretoryoviendo,'broad_celltypes', method='wilcoxon')
sc.tl.dendrogram(secretoryoviendo, groupby='broad_celltypes')
sc.pl.rank_genes_groups_heatmap(secretoryoviendo, n_genes=25, swap_axes=True, show_gene_labels=False,
                                vmin=-3, vmax=3 ,save ='secretoryphaseoviendoDEGS.png')